In [1]:
import pandas as pd
import errno    
import os
import numpy as np
import math

from sklearn.metrics import r2_score
from scipy import stats

from sklearn.metrics import mean_squared_error


'''
Author: Enrico Ceccolini
    TODO
'''

datadir = "/datasets/eurora_data/db1/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "april_long_jobs.csv"

## 3 settings Alina
#interval_comment = "Alina"
#start_time = pd.to_datetime('2014-06-30')
#end_time = pd.to_datetime('2014-11-01')

infile_long_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"
infile_short_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_short_jobs_real_pow"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']

In [2]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

long_jobs = pd.read_csv(infile_long_jobs + ".csv", index_col=0)
print("There are {} long jobs in the {}'s interval".format(long_jobs.shape[0], interval_comment))

### clean the data
# remove jobs runned on the inexistent node 129
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129] 
# remove jobs with the same id that runned on the same node
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("after the clean, jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

### merge the info from the two tables
merged_jobs_to_nodes = pd.merge(long_jobs, jobs_to_nodes_whole_data, how='left', on='job_id_string')
print("merged_jobs_to_nodes contains {} records".format(merged_jobs_to_nodes.shape[0]))

jobs_to_nodes_whole_data contains 469095 records


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are 35284 long jobs in the Andrea's interval
after the clean, jobs_to_nodes_whole_data contains 444610 records
merged_jobs_to_nodes contains 41216 records


In [3]:
#short_jobs = pd.read_csv(infile_short_jobs + ".csv", index_col=0)
#print("There are {} long jobs in the {}'s interval".format(short_jobs.shape[0], interval_comment))

In [4]:
merged_jobs_to_nodes.shape[0]

41216

# Calculate the mean core consumption

In [5]:
for node in nodes:
    #print(node)
    jobs_to_node = merged_jobs_to_nodes[merged_jobs_to_nodes['node_id'] == int(node)]
    #print(jobs_to_node.shape[0])
    mean_core_consumption = jobs_to_node['real_pow'].sum() / jobs_to_node['cpu_req'].sum()
    jobs_to_node = jobs_to_node[jobs_to_node['real_pow_quality'] != 0]
    mean_core_consumption_medium = jobs_to_node['real_pow'].sum() / jobs_to_node['cpu_req'].sum()
    jobs_to_node = jobs_to_node[jobs_to_node['real_pow_quality'] >= 0.5]
    mean_core_consumption_high = jobs_to_node['real_pow'].sum() / jobs_to_node['cpu_req'].sum()
    print("{}, {}, {}, {}".format(node, mean_core_consumption, mean_core_consumption_medium, mean_core_consumption_high))

01, 9.303076930042387, 10.074514527415673, 11.298611600237855
02, 7.697023455152835, 7.702996341486928, 8.34226259206578
03, 13.797005282298862, 13.797005282298862, 13.835960591081806
04, 13.935839123447922, 13.951987256847051, 13.951987256847051
05, 13.257317868065373, 13.275902348060043, 13.341967313787341
06, 12.08636545118669, 12.093847871072398, 12.646566358026407
07, 12.968377460177507, 12.998901564585092, 13.08164555023053
08, 13.021782963872438, 13.034363991131361, 13.563582129415245
09, 11.193661332340675, 11.219268561871193, 11.696159419999914
10, 11.668800701842088, 11.699855851828849, 12.067063157530551
11, 12.710658027501301, 12.743872513783613, 12.73582842592844
12, 12.046880288668913, 12.066747653234051, 12.448287945033789
13, 11.88099906422749, 11.907813454714653, 12.352682667831084
14, 12.37029828790492, 12.395948407564886, 12.826790898566914
15, 11.423465760151599, 11.460227775069091, 11.75988419493688
16, 11.23157027230262, 11.23157027230262, 11.567829243659743
17, 1

## Predict

In [4]:
infile_mean_core_consumption_perNode = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_consumption_perNode"
mean_core_consumption_perNode = pd.read_csv(infile_mean_core_consumption_perNode + ".csv")
mean_core_consumption_perNode.head(35)

,node,all,medium,high
0,1,9.303077,10.074515,11.298612
1,2,7.697023,7.702996,8.342263
2,3,13.797005,13.797005,13.835961
3,4,13.935839,13.951987,13.951987
4,5,13.257318,13.275902,13.341967
5,6,12.086365,12.093848,12.646566
6,7,12.968377,12.998902,13.081646
7,8,13.021783,13.034364,13.563582
8,9,11.193661,11.219269,11.696159
9,10,11.668801,11.699856,12.067063


In [7]:
merged_jobs_to_nodes.head(1)

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,n_cores_2_2,n_cores_3_1,user_short,pred_user,pred_userJob,node_id,ncpus,ngpus,nmics,mem_requested
0,498458.node129,rth2e6t0,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000@node129.eurora.cineca.it,1,16,14,...,16.0,0.0,alupi000,225.932207,198.527435,17,16,0,0,14680064


In [8]:
# high
long_jobs['pred_perNode'] = 0.0
i = 0

for index_job, job in long_jobs.iterrows():

    job_to_nodes = merged_jobs_to_nodes[merged_jobs_to_nodes['job_id_string'] == job['job_id_string']] 
    job_predicted_consumption = 0.0
    
    for index_node, job_to_node in job_to_nodes.iterrows():
        mean_core_consumption = mean_core_consumption_perNode[mean_core_consumption_perNode['node'] == job_to_node['node_id']].iloc[0][3]
        job_predicted_consumption += job_to_node['ncpus'] * mean_core_consumption
        
    long_jobs.at[index_job, 'pred_perNode'] = job_predicted_consumption
    if (i % 10000 == 0):
        print(i)
    i += 1



0
10000
20000
30000


In [9]:
long_jobs.to_csv(infile_long_jobs + ".csv")

# Validate

In [3]:
good_quality_jobs = long_jobs[long_jobs['real_pow_quality'] != 0]
#print(math.sqrt(np.var(good_quality_jobs['real_pow'])))
#good_quality_jobs = long_jobs

mean_real_pow = long_jobs['real_pow'].mean()
print("the mean is {}".format(mean_real_pow))


good_quality_jobs['numerator'] = (good_quality_jobs['real_pow'] - good_quality_jobs['pred_perNode'])**2
good_quality_jobs['denumerator'] = (good_quality_jobs['real_pow'] - mean_real_pow)**2
N = good_quality_jobs['numerator'].shape[0]

r_2 = 1 - (good_quality_jobs['numerator'].sum() / good_quality_jobs['denumerator'].sum())
print("r_2 is {}".format(r_2))

rmse = math.sqrt(good_quality_jobs['numerator'].sum() / N)
print("RMSE is {}".format(rmse))
print("NRMSE is {}".format(rmse/mean_real_pow))

the mean is 110.35204538702968
r_2 is 0.7152788986460095
RMSE is 143.91122321157286
NRMSE is 1.304110156788155


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [8]:
#NRMSE
good_quality_jobs['variance'] = (good_quality_jobs['real_pow'] - good_quality_jobs['pred_perNode'])**2
math.sqrt(good_quality_jobs['variance'].sum() / good_quality_jobs.shape[0]) / good_quality_jobs['real_pow'].mean()

1.2827469327989323

In [9]:
# R^2
good_quality_jobs['variance_2'] = (good_quality_jobs['real_pow'] - good_quality_jobs['real_pow'].mean())**2
1-(good_quality_jobs['variance'].sum() / good_quality_jobs['variance_2'].sum())   

0.7095656556763021

In [19]:
good_quality_jobs = long_jobs[long_jobs['real_pow_quality'] > 0.5]

slope, intercept, r_value, p_value, std_err = stats.linregress(good_quality_jobs['real_pow'], good_quality_jobs['pred_perNode'])
print("R-squared: {}".format(r_value**2))
print("mean_squared_error: {}".format(mean_squared_error(good_quality_jobs['real_pow'], good_quality_jobs['pred_perNode'])))
print(math.sqrt(mean_squared_error(good_quality_jobs['real_pow'], good_quality_jobs['pred_perNode'])))

R-squared: 0.7328285526560071
mean_squared_error: 19990.772972388608
141.38873000486498
